In [1]:
import pandas as pd
import numpy as np

In [3]:
# Source: https://www.kaggle.com/c/twitter-sentiment-analysis2/data
# train: https://www.kaggle.com/c/6740/download/train.csv
# test: https://www.kaggle.com/c/6740/download/test.csv

df = pd.read_csv('train.csv', encoding='latin-1')

In [4]:
X = df['SentimentText'].values
y = df['Sentiment'].values

In [5]:
from keras.preprocessing.text import Tokenizer

tk = Tokenizer()
tk.fit_on_texts(X)

/home/richard/dev/Twitter-sentiment-analysis/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
len(tk.word_index)

106612

In [7]:
max_words = 10000
tk.num_words = max_words
X_seq = tk.texts_to_sequences(X)

In [8]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 30
X_pad = pad_sequences(X_seq, maxlen=maxlen)

In [9]:
X_pad.shape

(99989, 30)

In [10]:
y.shape

(99989,)

In [11]:
def one_hot_encode_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [12]:
X_oht = one_hot_encode_sequences(X_pad, dimension=max_words)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_oht, y, test_size=0.33, random_state=0)

In [15]:
X_train.shape

(66992, 10000)

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
def model_1():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(max_words,)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    model.summary()
    
    return model

In [20]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

checkpointer = ModelCheckpoint('model-{epoch:02d}-{val_acc:.2f}.hdf5', verbose=1)
earlystopper = EarlyStopping(patience=2, verbose=1)

model = model_1()
history = model.fit(X_train, y_train, 
                    validation_split=0.2, 
                    batch_size=32, epochs=10, 
                    verbose=2, callbacks=[checkpointer, earlystopper])
# peak at (9, 0.7684)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 320,065
Trainable params: 320,065
Non-trainable params: 0
_________________________________________________________________
Train on 53593 samples, validate on 13399 samples
Epoch 1/10
Epoch 00001: saving model to model-01-0.76.hdf5
 - 9s - loss: 0.5269 - acc: 0.7417 - val_loss: 0.4998 - val_acc: 0.7619
Epoch 2/10
Epoch 00002: saving model to model-02-0.77.hdf5
 - 8s - loss: 0.4742 - acc: 0.7796 - val_loss: 0.4980 - val_acc: 0.7696
Epoch 3/10
Epoch 00003: saving model to model-03-0.77.hdf5
 - 8s - loss: 0.4613 - acc: 0.7897 - val_loss: 0.4945 - val_acc: 0.7701
Epoch 4/10
Epoch 00004: saving model to model-04-0.77.hdf5
 - 8s - loss: 0.4528 - acc: 0.